In [1]:
! pip install py2neo
! pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 2.7 MB/s eta 0:00:00a 0:00:01
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [2]:
from py2neo import Graph, Node, Subgraph
import pandas as pd
import os

# ---------------- 配置 ----------------
URI  = "neo4j+s://6b8efb85.databases.neo4j.io"
USER = "neo4j"
PWD  = "Rff9aVKhPNbsClQIQaqbtPS26AeHmJTSzSKMRKzFHj0"   
graph = Graph(URI, auth=(USER, PWD))

# 1️⃣ 清空数据库
graph.run("MATCH (n) DETACH DELETE n")
print("数据清除成功！")

数据清除成功！


In [5]:
from py2neo import Graph, Node, Subgraph
import pandas as pd
import os

# ---------------- 配置 ----------------
URI  = "neo4j+s://6b8efb85.databases.neo4j.io"
USER = "neo4j"
PWD  = "Rff9aVKhPNbsClQIQaqbtPS26AeHmJTSzSKMRKzFHj0"  
graph = Graph(URI, auth=(USER, PWD))

# 1️⃣ 清空数据库
graph.run("MATCH (n) DETACH DELETE n")

# 2️⃣ 节点文件映射（**已补充 order_nodes.csv**）
node_files = {
    "supplier_nodes.csv": "Supplier",
    "shipper_nodes.csv":  "Shipper",
    "review_nodes.csv":   "Review",
    "product_nodes.csv":  "Product",
    "employee_nodes.csv": "Employee",
    "customer_nodes.csv": "Customer",
    "category_nodes.csv": "Category",
    "order_nodes.csv":    "Order"        # ← 新增
}

csv_dir = "/root/autodl-tmp/kefu/rag/graphrag_2.1.0/graphrag/origin_data/data/ceshi"
node_cache = {}     # label -> {id -> node}

# ---------- 通用读节点 ----------
for file_name, label in node_files.items():
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    nodes = []
    for _, row in df.iterrows():
        id_col = next(c for c in df.columns if ":ID" in c)
        nid = str(row[id_col])
        props = {"id": nid}
        for col in df.columns:
            if ":ID" in col or ":LABEL" in col or col == "labels:LABEL":
                continue
            key = col.split(":")[0]
            props[key] = str(row[col])
        node = Node(label, **props)
        nodes.append(node)
    graph.create(Subgraph(nodes))
    node_cache[label] = {n["id"]: n for n in nodes}
    print(f"✅ {label}: {len(nodes)} 节点已导入")

# 3️⃣ 边文件配置
edge_files = [
    ("review_product_edges.csv",      "Review",   "Product",  "ABOUT"),
    ("product_supplier_edges.csv",    "Product",  "Supplier", "SUPPLIED_BY"),
    ("customer_order_edges.csv",      "Customer", "Order",    "PLACED"),
    ("product_category_edges.csv",    "Product",  "Category", "BELONGS_TO"),
    ("order_shipper_edges.csv",       "Order",    "Shipper",  "SHIPPED_VIA"),
    ("order_product_edges.csv",       "Order",    "Product",  "CONTAINS"),
    ("employee_reports_to_edges.csv", "Employee", "Employee", "REPORTS_TO"),
    ("employee_order_edges.csv",      "Employee", "Order",    "PROCESSED"),
    ("customer_review_edges.csv",     "Customer", "Review",   "WROTE")
]

# ---------- 通用读边 ----------
for file_name, start_lbl, end_lbl, default_rel in edge_files:
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    start_col = next(c for c in df.columns if ":START" in c)
    end_col   = next(c for c in df.columns if ":END" in c)
    type_col  = ":TYPE" if ":TYPE" in df.columns else None

    rels = []
    for _, row in df.iterrows():
        sid  = str(row[start_col])
        eid  = str(row[end_col])
        rtyp = str(row[type_col]) if type_col else default_rel
        # 用缓存字典直接取节点
        s_node = node_cache[start_lbl].get(sid)
        e_node = node_cache[end_lbl].get(eid)
        if s_node and e_node:
            rels.append({"sid": sid, "eid": eid, "type": rtyp})

    if rels:
        graph.run(
            """
            UNWIND $rels AS r
            MATCH (a {id: r.sid}), (b {id: r.eid})
            CALL apoc.merge.relationship(a, r.type, {}, {}, b) YIELD rel as _
            RETURN count(_)
            """,
            rels=rels
        )
    print(f"✅ {file_name}: {len(rels)} 条关系已创建")

print("🎉 全部节点与关系导入完成！")

✅ Supplier: 15 节点已导入
✅ Shipper: 10 节点已导入
✅ Review: 30 节点已导入
✅ Product: 10 节点已导入
✅ Employee: 3 节点已导入
✅ Customer: 20 节点已导入
✅ Category: 20 节点已导入
✅ Order: 100 节点已导入
✅ review_product_edges.csv: 30 条关系已创建
✅ product_supplier_edges.csv: 10 条关系已创建
✅ customer_order_edges.csv: 100 条关系已创建
✅ product_category_edges.csv: 10 条关系已创建
✅ order_shipper_edges.csv: 100 条关系已创建
✅ order_product_edges.csv: 303 条关系已创建
✅ employee_reports_to_edges.csv: 0 条关系已创建
✅ employee_order_edges.csv: 100 条关系已创建
✅ customer_review_edges.csv: 30 条关系已创建
🎉 全部节点与关系导入完成！
